In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
%matplotlib notebook
plt.style.use('seaborn-notebook')

In [3]:
data_2 = pd.read_csv(r'C:\Users\monis\Desktop\ML-Ang\Datasets\Health_Insurance_Lead_Prediction\test_YCcRUnU.csv')
test = pd.DataFrame(data_2)

In [4]:
for i in test.columns:
    print(i,test[i].unique(),'\n')

ID [50883 50884 50885 ... 72685 72686 72687] 

City_Code ['C1' 'C4' 'C3' 'C2' 'C13' 'C18' 'C16' 'C17' 'C11' 'C7' 'C21' 'C34' 'C14'
 'C8' 'C12' 'C6' 'C10' 'C9' 'C23' 'C32' 'C26' 'C15' 'C24' 'C20' 'C19' 'C5'
 'C28' 'C30' 'C25' 'C29' 'C27' 'C33' 'C22' 'C36' 'C35' 'C31'] 

Region_Code [ 156    7  564 ... 4778 4555 5204] 

Accomodation_Type ['Owned' 'Rented'] 

Reco_Insurance_Type ['Individual' 'Joint'] 

Upper_Age [30 69 28 23 75 24 56 41 22 38 42 31 35 46 44 29 60 54 27 71 68 55 36 59
 72 33 47 63 45 66 26 67 64 57 51 37 70 25 65 34 21 52 61 58 49 43 48 40
 50 39 32 53 62 19 20 18 74 73] 

Lower_Age [30 68 28 23 75 24 56 41 22 37 38 42 31 35 44 29 60 54 27 71 66 36 26 47
 63 45 59 67 64 57 43 46 61 25 70 69 65 72 55 34 21 52 19 58 33 53 49 48
 40 50 39 32 17 51 74 62 20 18 73 16] 

Is_Spouse ['No' 'Yes'] 

Health Indicator [nan 'X1' 'X3' 'X2' 'X5' 'X4' 'X7' 'X6' 'X9' 'X8'] 

Holding_Policy_Duration ['6.0' '3.0' '2.0' nan '14+' '5.0' '1.0' '4.0' '12.0' '11.0' '7.0' '9.0'
 '13.0' '8.0' '14.

In [5]:
test['City_Code'] = test['City_Code'].str.lstrip('C')
test['City_Code'] = test['City_Code'].astype(int)
test['City_Code']

0         1
1         4
2         1
3         3
4         1
         ..
21800     3
21801     4
21802    12
21803    10
21804     2
Name: City_Code, Length: 21805, dtype: int32

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21805 entries, 0 to 21804
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       21805 non-null  int64  
 1   City_Code                21805 non-null  int32  
 2   Region_Code              21805 non-null  int64  
 3   Accomodation_Type        21805 non-null  object 
 4   Reco_Insurance_Type      21805 non-null  object 
 5   Upper_Age                21805 non-null  int64  
 6   Lower_Age                21805 non-null  int64  
 7   Is_Spouse                21805 non-null  object 
 8   Health Indicator         16778 non-null  object 
 9   Holding_Policy_Duration  13202 non-null  object 
 10  Holding_Policy_Type      13202 non-null  float64
 11  Reco_Policy_Cat          21805 non-null  int64  
 12  Reco_Policy_Premium      21805 non-null  float64
dtypes: float64(2), int32(1), int64(5), object(5)
memory usage: 2.1+ MB


In [7]:
test.isna().sum()

ID                            0
City_Code                     0
Region_Code                   0
Accomodation_Type             0
Reco_Insurance_Type           0
Upper_Age                     0
Lower_Age                     0
Is_Spouse                     0
Health Indicator           5027
Holding_Policy_Duration    8603
Holding_Policy_Type        8603
Reco_Policy_Cat               0
Reco_Policy_Premium           0
dtype: int64

In [8]:
df4 = test.copy()

In [9]:
bin_premium_group = [0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, df4['Reco_Policy_Premium'].max()]
label_bin = ['Basic-1', 'Basic-2', 'Medium-1', 'Meduim-2', 'High-1', 'High-2','Gold', 'Platinum', 'Diamond']

df4['Premium_group'] = pd.cut(df4['Reco_Policy_Premium'], bins = bin_premium_group, labels = label_bin)
df4.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Premium_group
0,50883,1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0,Medium-1
1,50884,4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8,Gold
2,50885,1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0,Basic-2
3,50886,3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0,Basic-2
4,50887,1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0,High-1


In [10]:
bin_age_generation = [15, 34, 50, 69, df4['Upper_Age'].max()]
label_age_generation = ['Millenials', 'Gen X', 'Boomer', 'Silent']

df4['Generation-1'] = pd.cut(df4['Upper_Age'], bins = bin_age_generation, labels = label_age_generation)
df4['Generation-2'] = pd.cut(df4['Lower_Age'], bins = bin_age_generation, labels = label_age_generation)
df4.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Premium_group,Generation-1,Generation-2
0,50883,1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0,Medium-1,Millenials,Millenials
1,50884,4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8,Gold,Boomer,Boomer
2,50885,1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0,Basic-2,Millenials,Millenials
3,50886,3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0,Basic-2,Millenials,Millenials
4,50887,1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0,High-1,Silent,Silent


In [11]:
df4.dtypes

ID                            int64
City_Code                     int32
Region_Code                   int64
Accomodation_Type            object
Reco_Insurance_Type          object
Upper_Age                     int64
Lower_Age                     int64
Is_Spouse                    object
Health Indicator             object
Holding_Policy_Duration      object
Holding_Policy_Type         float64
Reco_Policy_Cat               int64
Reco_Policy_Premium         float64
Premium_group              category
Generation-1               category
Generation-2               category
dtype: object

In [12]:
df4['Is_Spouse'] = df4['Is_Spouse'].map({'Yes':1, 'No':0})
df4['Accomodation_Type'] = df4['Accomodation_Type'].map({'Rented':1, 'Owned':2})
df4['Reco_Insurance_Type'] = df4['Reco_Insurance_Type'].map({'Individual':1, 'Joint':2})

In [13]:
df4['Holding_Policy_Duration'].fillna('0.0')

0        6.0
1        3.0
2        2.0
3        3.0
4        0.0
        ... 
21800    4.0
21801    6.0
21802    0.0
21803    3.0
21804    14+
Name: Holding_Policy_Duration, Length: 21805, dtype: object

In [14]:
df4['Health Indicator'].fillna('X0')
df4['Health Indicator'] = df4['Health Indicator'].str.lstrip('X')
df4['Health Indicator'] = df4['Health Indicator'].astype(int)


ValueError: cannot convert float NaN to integer

In [19]:
df4['Health Indicator'].fillna(0)

0        0
1        1
2        3
3        3
4        3
        ..
21800    1
21801    5
21802    3
21803    4
21804    2
Name: Health Indicator, Length: 21805, dtype: object

In [20]:
df4['Premium_group'] = df4['Premium_group'].map({'Basic-1':1, 'Basic-2':2 , 'Medium-1':3, 'Meduim-2':4, 'High-1':5, 'High-2':6, 'Gold':7, 'Platinum':8, 'Diamond':9})

In [21]:
df4['Generation-1'] = df4['Generation-1'].map({'Millenials':1, 'Gen X':2, 'Boomer':3, 'Silent':4})
df4['Generation-2'] = df4['Generation-2'].map({'Millenials':1, 'Gen X':2, 'Boomer':3, 'Silent':4})

In [22]:
df4.fillna({'Holding_Policy_Duration':0, 'Health Indicator':0, 'Holding_Policy_Type':0}, inplace=True)
df4.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Premium_group,Generation-1,Generation-2
0,50883,1,156,2,1,30,30,0,0,6.0,3.0,5,11934.0,3,1,1
1,50884,4,7,2,2,69,68,1,1,3.0,3.0,18,32204.8,7,3,3
2,50885,1,564,1,1,28,28,0,3,2.0,4.0,17,9240.0,2,1,1
3,50886,3,1177,1,1,23,23,0,3,3.0,3.0,18,9086.0,2,1,1
4,50887,1,951,2,1,75,75,0,3,0,0.0,5,22534.0,5,4,4


In [23]:
df4['Health Indicator'] = pd.to_numeric(df4['Health Indicator'])

In [24]:
from sklearn.preprocessing import LabelEncoder
labelEncoder= LabelEncoder()
df4['Holding_Policy_Duration'] = labelEncoder.fit_transform(df4['Holding_Policy_Duration'].astype(str))


In [25]:
df5 = pd.get_dummies(df4, columns=['Generation-1', 'Generation-2', 'Premium_group'])

In [26]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21805 entries, 0 to 21804
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       21805 non-null  int64  
 1   City_Code                21805 non-null  int32  
 2   Region_Code              21805 non-null  int64  
 3   Accomodation_Type        21805 non-null  int64  
 4   Reco_Insurance_Type      21805 non-null  int64  
 5   Upper_Age                21805 non-null  int64  
 6   Lower_Age                21805 non-null  int64  
 7   Is_Spouse                21805 non-null  int64  
 8   Health Indicator         21805 non-null  int64  
 9   Holding_Policy_Duration  21805 non-null  int32  
 10  Holding_Policy_Type      21805 non-null  float64
 11  Reco_Policy_Cat          21805 non-null  int64  
 12  Reco_Policy_Premium      21805 non-null  float64
 13  Generation-1_1           21805 non-null  uint8  
 14  Generation-1_2        

In [27]:
df5['Response'] = 0

In [28]:
df5.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Generation-1_1',
       'Generation-1_2', 'Generation-1_3', 'Generation-1_4', 'Generation-2_1',
       'Generation-2_2', 'Generation-2_3', 'Generation-2_4', 'Premium_group_1',
       'Premium_group_2', 'Premium_group_3', 'Premium_group_4',
       'Premium_group_5', 'Premium_group_6', 'Premium_group_7',
       'Premium_group_8', 'Premium_group_9', 'Response'],
      dtype='object')

In [29]:
df5 = df5[['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Generation-1_1',
       'Generation-1_2', 'Generation-1_3', 'Generation-1_4', 'Generation-2_1',
       'Generation-2_2', 'Generation-2_3', 'Generation-2_4','Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat',  'Premium_group_1',
       'Premium_group_2', 'Premium_group_3', 'Premium_group_4',
       'Premium_group_5', 'Premium_group_6', 'Premium_group_7',
       'Premium_group_8', 'Premium_group_9', 'Response']]

In [37]:
df5.isna().sum()

ID                         0
City_Code                  0
Region_Code                0
Accomodation_Type          0
Reco_Insurance_Type        0
Generation-1_1             0
Generation-1_2             0
Generation-1_3             0
Generation-1_4             0
Generation-2_1             0
Generation-2_2             0
Generation-2_3             0
Generation-2_4             0
Is_Spouse                  0
Health Indicator           0
Holding_Policy_Duration    0
Holding_Policy_Type        0
Reco_Policy_Cat            0
Premium_group_1            0
Premium_group_2            0
Premium_group_3            0
Premium_group_4            0
Premium_group_5            0
Premium_group_6            0
Premium_group_7            0
Premium_group_8            0
Premium_group_9            0
Response                   0
dtype: int64

In [ ]:
import pickle
Pkl_Filename = "Pickle_RL_Model.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    Pickled_RF_Model = pickle.load(file)

Pickled_RF_Model

In [ ]:
final_test = df5.drop('Response', axis=1)
test_prediction = Pickled_RF_Model.predict(final_test)
print(test_prediction)
test_prediction.shape

In [ ]:
test_prediction.sum()

In [ ]:
submission_df = pd.DataFrame(final_test['ID'], columns=['ID'])
submission_df['Response'] = test_prediction 

In [ ]:
submission_df.to_csv('C:\\Users\\monis\\Desktop\\ML-Ang\\Datasets\\Health_Insurance_Lead_Prediction\\submission_03.csv', index=False)

In [39]:
import pickle
Pkl_Filename = "Pickle_RF1_Model.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    Pickled_RF1_Model = pickle.load(file)

Pickled_RF1_Model

RandomForestClassifier()

In [40]:
final_test = df5.drop(['Response','ID'], axis=1)
test_prediction = Pickled_RF1_Model.predict(final_test)
print(test_prediction)
test_prediction.shape

[1 1 0 ... 1 0 0]


(21805,)

In [41]:
test_prediction.sum()

11482

In [43]:
submission_df = pd.DataFrame(df5['ID'], columns=['ID'])
submission_df['Response'] = test_prediction 
submission_df.to_csv('C:\\Users\\monis\\Desktop\\ML-Ang\\Datasets\\Health_Insurance_Lead_Prediction\\submission_03.csv', index=False)